<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_BERT_inference_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference

Let's test the model on new datapoints:

In [1]:
# Do not use conda in Colab notebook, do not try to install with conda, use pip

!pip install -q huggingface_hub
!pip install -q datasets

import json
import os
import pandas as pd
import torch

from datasets        import DatasetDict
from google.colab    import files, userdata
from huggingface_hub import login, hf_hub_download
from transformers    import AutoModelForSequenceClassification, AutoTokenizer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
### Check the device: no need to use T4 GPU for inference

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")


device: cuda


In [3]:
### Hugging Face authenticate

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
hf_token               = os.environ.get('HF_TOKEN')
login(token = hf_token)

# Verify
!huggingface-cli whoami


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [4]:
### Model repo name on Hugging Face Hub

HF_Hub_name = "claudelepere/skill_classification"


In [5]:
### Download the labels from the dataset repo

file_path = hf_hub_download(
    repo_id                = HF_Hub_name,
    filename               = "labels.json",
    repo_type              = "dataset",
    local_dir              = "./"
    )

with open(file_path, 'r') as f:
    labels = json.load(f)

print(f"labels: {type(labels)} len={len(labels)}\n{labels}")


labels.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

labels: <class 'list'> len=6
['135', '136', '137', '138', '139', '390']


In [6]:
### Get the filtered skills, those present in labels, in order to have their Value

files.upload()

skill_df          = pd.read_csv('skills.csv')
skill_df['Id']    = skill_df['Id'].astype(str)
skill_df['Value'] = skill_df['Value'].astype(str)
labels            = [str(label) for label in labels]
filtered_skill_df = skill_df[skill_df['Id'].isin(labels)]

print(f"filtered_skill_df: {type(filtered_skill_df)} shape={filtered_skill_df.shape}\n{filtered_skill_df}")


Saving skills.csv to skills.csv
filtered_skill_df: <class 'pandas.core.frame.DataFrame'> shape=(6, 3)
      Id  SkillTypeId                           Value
299  135           11                       First job
300  136           11  EXPERIENCE_6_MONTHS_TO_2_YEARS
301  137           11         EXPERIENCE_2_TO_5_YEARS
302  138           11    EXPERIENCE_MORE_THAN_5_YEARS
303  139           11   EXPERIENCE_MORE_THAN_10_YEARS
304  390           11              LESS_THAN_6_MONTHS


In [7]:
def skill_id2value(skill_id):
  return filtered_skill_df.loc[filtered_skill_df['Id'] == skill_id, 'Value'].values[0]


In [8]:
### Download the model and the tokenizer from the model repo

model = AutoModelForSequenceClassification.from_pretrained(
    HF_Hub_name,
    num_labels = len(labels)
)

tokenizer = AutoTokenizer.from_pretrained(HF_Hub_name)


config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [9]:
### New texts to classify

text_None = "I'm happy, I can finally train a model for multi-label classification."

# NL
text_323697 = "Talencia Consulting - Senior Full Stack Developer (Java & Angular) Full Stack, DevOps, AKS, OAuth, API Talencia Consulting Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,"
# 142,189,190,754,208,794,676,811,812,139,138
# SkillTypeId 7 : true: 142                preds: 142, 170
# SkillTypeId 8 : true: 189, 190, 754, 208 preds:
# SkillTypeId 9 : true: 676, 811, 812      preds:
# SkillTypeId 10: true: 794                preds:
# SkillTypeId 11: true: 139, 138           preds: 138, 139

# EN
text_23553 = "Cream Consulting - Business Analyst Insurance, Property&Casualty (P&C) Cream Consulting Today we are looking for Business Analyst to extend our team specialized in Business Process Improvement. You will join a multi-lingual team focusing on P&C business. In that position, you will define and clarify the role and responsibilities. You will be responsible for leading business process reviews where you will identify and implement innovative solutions. You will define and document the new processes and systems to meet our client's business objectives. What the job is all about? Implement new solutions for P&C insurance companies Write and validate the “as is” Plan the analysis of organization's strategic business needs Understand End Users needs Develop process modeling and design Follow up projects and monitor development Organize and conduct testing What Cream is looking for? At least 5 years of experience in Business Analysis in insurance business Analysis know-how Fluent editorial Good knowledge of Property & Casualty insurance business to interface with all stakeholders Functional knowledge of insurance products Good communication skills Fluent English, good knowledge in French and/or Dutch is a plus"
#356,168,149,795,802,137,139,138,353
# SkillTypeId 7 : true: 356, 168, 149 preds: 142, 170
# SkillTypeId 8 :
# SkillTypeId 9 :
# SkillTypeId 10: true: 795, 802
# SkillTypeId 11: true: 137, 139, 138 preds: 138, 139
# SkillTypeId 12: true: 353

text_323611 = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
# "171,170,794,800,798,797,138,139,352"
# SkillTypeId 7 : true: 170, 171           preds: 142, 170
# SkillTypeId 8 :
# SkillTypeId 9 :
# SkillTypeId 10: true: 794, 800, 798, 797 preds:
# SkillTypeId 11: 138, 139                 preds: 138, 139
# SkillTypeId 12: 352

text_323526 = "Vivid Resourcing - Chief Technology Officer CTO, reliability, business goals Vivid Resourcing We're partnered with a leading sustainability-oriented company near Brussels, aiming to combat high pollution rates worldwide. They are currently working on a unique application that rewards workers for reducing their carbon footprint, whilst also maintaining and even improving profits. Together we are seeking a visionary Chief Technology Officer (CTO) who aligns with their mission and ambitions. The ideal candidate will possess a strong hands-on technical background, proven management experience, and strong business acumen. This role requires a strategic thinker who can drive technological direction and support the company's growth objectives of transitioning from a scale-up to an established business entity, so any past experience leading teams in this manner would go a long way. Key responsibilities Develop and execute the company's technological vision and strategy Lead and mentor a team of engineers and technologists Oversee all technical aspects of the company, ensuring alignment with business goals Drive innovation in regenerative sustainable technologies and carbon measurement systems Collaborate with cross-functional teams to integrate technology solutions Ensure the reliability, security, and scalability of technological infrastructures Foster a culture of continuous improvement and technical excellence Qualifications Experience leading a team within a small to medium sized company Strong technical background in software development/data analytics/system architecture Bachelor's or Master's degree in either an IT or Business related field Experience in the agriculture or environmental sectors is a plus Proven management skills with the ability to lead, communicate and inspire a diverse team Excellent business acumen and strategic thinking Strong problem-solving skills and the ability to make informed decisions in a fast-paced environment Offer Taking charge of a genuinely impactful product, using your direction for the good of the environment Complete responsibility over a technical team, with management responsibilities Up to 110,000 EUR gross for experienced applicants, which can then be increased further Full benefits package including mobility costs Flexible hybrid work Inclusive work environment If this role interests you, attach a CV and apply today!"
# 175,169,139,352
# SkillTypeId 7 : true: 169, 175 preds: 142, 170
# SkillTypeId 8 : true:
# SkillTypeId 9 : true:
# SkillTypeId 10: true:
# SkillTypeId 11: true: 139      preds: 138, 139, 137
# SkillTypeId 12: true: 352

text_323408 = "Michael Page - Sales Engineer Financial Software, Outlook, Word Michael Page This company provides cutting-edge software solutions for the Finance industry, helping institutions manage investment portfolios, risk assessments, and regulatory compliance. Their user-friendly platform leverages advanced technology and data analytics to enhance decision-making and operational efficiency. They are looking for a Senior Sales to support a team of three people and offer a software solution to their clients. This company provides cutting-edge software solutions for the Finance industry, helping institutions manage investment portfolios, risk assessments, and regulatory compliance. Their user-friendly platform leverages advanced technology and data analytics to enhance decision-making and operational efficiency. Missions Actively oversees the team's efforts in identifying new prospects that are well-suited for the various modules of the software solution. Develops and nurtures a network of relationships to spot emerging market trends. Focuses on key European markets of interest, including Belgium, France, Switzerland, Luxembourg, and the Netherlands. Possesses a strong understanding of business needs and solutions. Has extensive knowledge of portfolio investment management. Sets pricing strategies for the Sales process. Meets Sales targets. Coordinates the Marketing and Communication plan. Defines and implements the prospecting strategy, ensuring follow-up with the Sales team. Collaborates effectively with other departments within the company. Profile Holds a Master's degree in Finance or Economics. At least 5 years of experience in selling financial software. Fluent in French and English (Dutch is a plus). Proficient in using Office tools (Outlook, Word, Excel, PowerPoint). Strong understanding of marketing activities. Willingness to travel within Europe. Adept at facing challenges. Skilled in teamwork and collaboration. Exhibits leadership qualities. Aligns with the company's values and is ready to promote them. Conditions and benefits An attractive salary bpackage including extra-legal benefits with bonus A friendly work environment in a dynamic company"
# "169,176,385,139,138,353"
# SkillTypeId 7 : true: 169, 176
# SkillTypeId 8 : true:
# SkillTypeId 9 : true: 385
# SkillTypeId 10: true:
# SkillTypeId 11: true: 139, 138    preds: 138, 139, 137
# SkillTypeId 12: true: 353

# FR
text_323517 = "AG Insurance - Technical Architect REST API, DevOps AG Insurance Un lieu de travail où vous pouvez prendre des initiatives. Où tout le monde vous encourage à montrer ce dont vous êtes capable. Où on vous encourage à vous développer. Et où vous construisez l'avenir avec vos collègues. Vous avez de l'audace et le client est au centre de vos préoccupations ? Alors, à très bientôt chez AG ! Notre compagnie compte environ 700 spécialistes IT. Autant de collègues géniaux qui contribuent chaque jour à la (r)évolution technologique chez AG. Le département Information System (IS) est responsable du développement des applications supportant le business assurance, ainsi que des applications transversales dans les domaines suivants : In & Outbounds, Referentials & Financials, sans oublier des centres d'expertise technologique pour le développement des Front-Ends, pour la mise en place d'APIs et s... Et si c'était votre prochain job ? Nous recherchons un(e) .NET Solution Architect pour renforcer ce département. Vous proposez des solutions techniques adaptées aux besoins et aux contraintes des projets et conformes à la vision IT d'AG. En innovant et en améliorant ce qui existe déjà, vous créez également une valeur ajoutée pour AG. Pendant la phase architecturale, vous choisissez les solutions en concertation avec les autres architectes techniques et fonctionnels, et avec toutes les autres parties concernées (autres équipes, partenaires externes, collègues d'Infrastructure & Operations, etc.). Vous concevez l'architecture technique des applications sur la base des exigences fonctionnelles et non fonctionnelles. Nous comptons également sur vous pour le support technique, du développement à la production de l'application. Vous assurez le suivi de l'implémentation de vos solutions, en coopération avec les équipes de développement. Vous vous assurez ensuite que la solution mise en œuvre est conforme à l'architecture proposée. Si nécessaire, vous apportez des corrections. Vous accompagnez et coachez vos collègues pour les aider à monter en compétence dans le domaine. Ce poste vous intéresse ? Qu'attendez-vous pour postuler ? Nos recruteurs examineront avec vous quelle division de notre département IT et quelle équipe vous conviendront le mieux. Vous vous reconnaissez dans ce profil ? Les nouvelles technologies ? Vous en suivez de près l'évolution et les tendances en architecture IT. Vous avez une expérience solide dans les domaines suivants : Technical Architecture & Design Patterns REST API, .NET Framework & .NET 6 and up Source control tooling like Azure DevOps & GitHub Microsoft Azure Cloud SQL Server Vous êtes autonome et prenez les choses en main ? Bien sûr ! Mais vous savez remonter les informations ou demander du support si nécessaire. Team-player dans l'âme, vous partagez naturellement votre savoir-faire avec vos collègues. La qualité du travail, dans ses moindres détails, c'est votre cheval de bataille. Vous le documentez avec précision, tant pour le support technique de votre équipe que pour les utilisateurs d'autres teams. Communiquer en français et/ou en néerlandais, à l'oral et à l'écrit ? Un jeu d'enfant pour vous ! Votre anglais est à la hauteur ? Encore mieux ! Un diplôme supérieur en informatique et/ou plus de 2 ans d'expérience dans une fonction similaire ? Foncez ! Vous n'allez pas rater ça ? Un super job chez le leader sur le marché de l'assurance. Et comme nous cherchons à encore mieux servir nos clients, nous comptons sur vous pour nous y aider. Un environnement de travail moderne dans tous les sens du terme : physique, digital et organisationnel. En d'autres termes, des heures de travail flexibles, la possibilité de télétravailler jusqu'à 3 jours par semaine et le matériel IT adéquat pour travailler de la maison. Une équipe enthousiaste et dynamique, notée 10/10 pour l'ambiance et la convivialité. La possibilité de vous perfectionner en continu, grâce à un large éventail de formations. Idéal pour apprendre toutes les compétences qui vous aideront à faire évoluer votre carrière. Une véritable carrière. En effet, travailler chez AG, c'est bien plus qu'un job. Envie d'explorer de nouveaux horizons après quelque temps ? Nous vous guidons et vous encourageons à exploiter tous vos talents à fond. Des bonnes vibrations, grâce à un vaste programme de bien-être bourré d'activités sportives et d'ateliers inspirants. De quoi vous sentir (encore) plus épanoui(e) dans votre travail et votre vie. Un lieu où tout le monde se sent bienvenu et a des chances égales de s'épanouir. Où votre avenir n'est pas déterminé par votre origine, votre âge, votre genre, votre orientation sexuelle ou votre handicap, mais par votre talent et vos compétences. Et comme l'argent, ça compte aussi : un package salarial attrayant. Vous pouvez même composer vous-même une partie de votre package, car personne ne sait mieux que vous ce dont vous avez besoin. Lancez-vous ! Emballé(e) ? > Postulez aujourd'hui encore ! Nous nous ferons un plaisir de vous aider si vous avez besoin de soutien avant ou pendant le processus de sélection."
# 146,636,676,668,670,300,812,138,137
# SkillTypeId 7       : true: 146 preds: 142, 170, 689
# SkillTypeId 8, 9, 10: true: 636, 676, 668, 670, 300, 812
# SkillTypeId 11      : true: 138, 137 preds: 137, 138


text = text_323526


In [10]:
### Classify
"""
inputs = tokenizer(
    text,
    return_tensors='pt',
    truncation=True,
    padding='max_length',
    max_length=512
)
"""

"\ninputs = tokenizer(\n    text,\n    return_tensors='pt',\n    truncation=True,\n    padding='max_length',\n    max_length=512\n)\n"

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [11]:
### Perform the forward pass: logits are raw scores for each label
"""
with torch.no_grad():
  outputs = model(**inputs)
  logits  = outputs.logits

#print(f"logits: {type(logits)} shape={logits.shape}\n{logits}")
"""

'\nwith torch.no_grad():\n  outputs = model(**inputs)\n  logits  = outputs.logits\n\n#print(f"logits: {type(logits)} shape={logits.shape}\n{logits}")\n'

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [12]:
### Squash logits to [0, 1] probabilities:
"""
probs = torch.sigmoid(logits)
#print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")

probs = probs.squeeze()
#print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
"""

'\nprobs = torch.sigmoid(logits)\n#print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")\n\nprobs = probs.squeeze()\n#print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")\n'

In [13]:
### Convert probabilities into binary predictions
"""
threshold      = 0.5
preds          = (probs >= threshold).int()
#print(f"preds: type={type(preds)} shape={preds.shape}\n{preds}")

sorted_indices = torch.argsort(probs, descending=True)
print(f"sorted_indices: {type(sorted_indices)} shape={sorted_indices.shape}\n{sorted_indices}")
sorted_probs   = probs[sorted_indices]
print(f"sorted_probs: {type(sorted_probs)} shape={sorted_probs.shape}\n{sorted_probs}")
sorted_labels  = [labels[i] for i in sorted_indices]
print(f"sorted_labels: {type(sorted_labels)} len={len(sorted_labels)}\n{sorted_labels}")
sorted_preds   = preds[sorted_indices]
print(f"sorted_preds: {type(sorted_preds)} shape={sorted_preds.shape}\n{sorted_preds}")

print(f"skill\tprob (thr={threshold})\tpred\tValue")
for label, prob, pred, in zip(sorted_labels, sorted_probs, sorted_preds):
  print(f"{label}\t{prob.item():.4f}\t\t{int(pred.item())}\t{skill_id2value(label)}")

print()

print(f"skill\tprob (thr={threshold})\tpred\tValue")
if (preds == 0).all():
  print("All predictions are 0, no skills found")
else:
  for label, prob, pred in zip(sorted_labels, sorted_probs, sorted_preds):
    if pred == 1:
      print(f"{label}\t{prob.item():.4f}\t\t{int(pred.item())}\t{skill_id2value(label)}")
"""

'\nthreshold      = 0.5\npreds          = (probs >= threshold).int()\n#print(f"preds: type={type(preds)} shape={preds.shape}\n{preds}")\n\nsorted_indices = torch.argsort(probs, descending=True)\nprint(f"sorted_indices: {type(sorted_indices)} shape={sorted_indices.shape}\n{sorted_indices}")\nsorted_probs   = probs[sorted_indices]\nprint(f"sorted_probs: {type(sorted_probs)} shape={sorted_probs.shape}\n{sorted_probs}")\nsorted_labels  = [labels[i] for i in sorted_indices]\nprint(f"sorted_labels: {type(sorted_labels)} len={len(sorted_labels)}\n{sorted_labels}")\nsorted_preds   = preds[sorted_indices]\nprint(f"sorted_preds: {type(sorted_preds)} shape={sorted_preds.shape}\n{sorted_preds}")\n\nprint(f"skill\tprob (thr={threshold})\tpred\tValue")\nfor label, prob, pred, in zip(sorted_labels, sorted_probs, sorted_preds):\n  print(f"{label}\t{prob.item():.4f}\t\t{int(pred.item())}\t{skill_id2value(label)}")\n\nprint()\n\nprint(f"skill\tprob (thr={threshold})\tpred\tValue")\nif (preds == 0).all()

In [14]:
#print("Notebook successfully completed.")

In [15]:
### Upload and unzip the dataset zip file

uploaded_files     = files.upload()
uploaded_file_name = list(uploaded_files.keys())[0]
print(f"uploaded_file_name: {uploaded_file_name} {len(uploaded_files)}")

!unzip {uploaded_file_name}

unzipped_file_dir_name = os.path.splitext(uploaded_file_name)[0]
print(f"unzipped_file_dir_name: {unzipped_file_dir_name}")

### Create the dataset

dataset = DatasetDict.load_from_disk(unzipped_file_dir_name)

print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")


### Create the label list and the id2label and label2id mappings.

"""
labels
    if dataset 7_1000_125_125  , 48 labels
    if dataset 7_128_18_54     , 42 labels
    if dataset 8910_1087_68_204, 206 labels
"""

labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")


### Tokenize 'text'

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples, indices):
  text = examples['text']    # Batch of texts

  encoding = tokenizer(
      text,                             # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = 512,
      return_tensors = 'pt'             # Return PyTorch tensors
      )

  # Create an empty label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Add labels to the encoding
  encoding['labels'] = labels_matrix
  #print(f"encoding['labels']: {encoding['labels']}")

  return encoding

### Create the encoded dataset

encoded_dataset = dataset.map(
    preprocess_data,
    batched        = True,
    remove_columns = dataset['train'].column_names,
    with_indices   = True
    )

example = encoded_dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

print()

print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
print(f"example['token_type_ids']: {type(example['token_type_ids'])} {len(example['token_type_ids'])}\n{example['token_type_ids']}")
print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

tokenizer.decode(example['input_ids'])

[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

"""Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html)."""

encoded_dataset.set_format("torch")    # Ensures correctness and compatibility with PyTorch pipelines



Saving dataset_11_11000.zip to dataset_11_11000.zip
uploaded_file_name: dataset_11_11000.zip 1
Archive:  dataset_11_11000.zip
   creating: dataset_11_11000/
   creating: dataset_8910_1087_68_204/
   creating: dataset_11_11000/test/
   creating: dataset_11_11000/train/
   creating: dataset_11_11000/validation/
  inflating: dataset_11_11000/dataset_dict.json  
  inflating: dataset_11_11000/test/data-00000-of-00001.arrow  
  inflating: dataset_11_11000/test/dataset_info.json  
  inflating: dataset_11_11000/test/state.json  
  inflating: dataset_11_11000/train/data-00000-of-00001.arrow  
  inflating: dataset_11_11000/train/dataset_info.json  
  inflating: dataset_11_11000/train/state.json  
  inflating: dataset_11_11000/validation/data-00000-of-00001.arrow  
  inflating: dataset_11_11000/validation/dataset_info.json  
  inflating: dataset_11_11000/validation/state.json  
   creating: dataset_8910_1087_68_204/test/
   creating: dataset_8910_1087_68_204/train/
   creating: dataset_8910_1087_

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/8800 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

Map:   0%|          | 0/1650 [00:00<?, ? examples/s]

example: <class 'dict'> dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'input_ids': [101, 1060, 15549, 28032, 1011, 2449, 20302, 2923, 9413, 2361, 2291, 2368, 1060, 15549, 28032, 25520, 9413, 10755, 2368, 2449, 20302, 2923, 3841, 15333, 12297, 3372, 12155, 8551, 20806, 15992, 29536, 2953, 21770, 20302, 23274, 7389, 3158, 2793, 3089, 3501, 10343, 21572, 9623, 5054, 1010, 21770, 8909, 4765, 23664, 7389, 3158, 12034, 15141, 8613, 5302, 12439, 28418, 10023, 14728, 2078, 4372, 21770, 12170, 14728, 2078, 3158, 2358, 11657, 17701, 2818, 4748, 25929, 18168, 2139, 3653, 9153, 7368, 4372, 8114, 2666, 8915, 15502, 17288, 2368, 1012, 15333, 2057, 8024, 2102, 6583, 25974, 2168, 2078, 2777, 2310, 2869, 5428, 12179, 3207, 21358, 9247, 15542, 18168, 1999, 27586, 11039, 8915, 1047, 3089, 3501, 6914, 1999, 15876, 2078, 3169, 12260, 2022, 14490, 6199, 2368, 4372, 21318, 2102, 2850, 4726, 2368, 1012, 15333, 2310, 13320, 2389, 2102, 2139, 4371, 1999, 27586, 11039, 2368, 6583, 2906,

In [17]:
print(model.device)
print(input_tensor.device)
print(label_tensor.device)


cpu


NameError: name 'input_tensor' is not defined

In [21]:
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

model = model.to(device)  # Ensure the model is on the same device


#encoded_dataset = DatasetDict.load_from_disk('encoded_dataset')
# Get predictions on the test set
model.eval()
with torch.no_grad():
  preds       = []
  true_labels = []
  for batch in encoded_dataset['test']:
    input_ids      = batch['input_ids'].unsqueeze(0).to(device)       # Add batch dimension
    attention_mask = batch['attention_mask'].unsqueeze(0).to(device)  # Add batch dimension
    labels         = batch['labels'].unsqueeze(0).to(device)          # Add batch dimension
    outputs        = model(input_ids, attention_mask=attention_mask)
    logits         = outputs.logits
    preds.append(torch.sigmoid(logits).cpu().numpy()) # Convert to Numpy
    true_labels.append(labels.cpu().numpy())          # Convert to Numpy

# Convert predictions and true labels to numpy arrays
preds       = np.concatenate(preds, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# Evaluate using metrics
threshold = 0.5
binary_preds = (preds >= threshold).astype(int)
print(f"preds: {type(preds)} shape={preds.shape}\n{preds}")
print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")
#print(classification_report(true_labels, binary_preds, target_names=labels))
#print(f"ROC AUC Score: {roc_auc_score(true_labels, preds, average='micro')}")


preds: <class 'numpy.ndarray'> shape=(1650, 6)
[[0.00376406 0.01381681 0.6339289  0.98549104 0.9401884  0.00312276]
 [0.02599069 0.56973976 0.9804827  0.8930631  0.13848796 0.04949472]
 [0.00328308 0.01427895 0.49755782 0.9866306  0.9557439  0.00336975]
 ...
 [0.04649998 0.7365664  0.97627074 0.8919156  0.2571542  0.10139058]
 [0.00663384 0.02944014 0.94618547 0.98861945 0.94610995 0.00753449]
 [0.00762244 0.03344294 0.9168479  0.9863235  0.9671783  0.00904374]]
true_labels: <class 'numpy.ndarray'> shape=(1650, 6)
[[0. 0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 ...
 [0. 0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 0.]]
